In [1]:
# !pip install git+https://github.com/XanaduAI/pennylane.git@prob_fn

In [2]:
import pennylane as qml
from variational_notebooks.entangler import get_entangler_map
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer
from pennylane.beta import prob


from data_loader import *

In [3]:
idx_array = []
X = []
Y = []

for idx, ((i, j), x, y) in enumerate(test_celeb_face_loader):
    idx_array.append((i, j))
    X.append(x.tolist())
    Y.append(y.item())
    

In [4]:
X = X[:5]
Y = Y[:5]

len(Y)

5

In [5]:
n_wires = 9
n_qubits_readout = 8
dev = qml.device("default.qubit", wires=n_wires)

graph = get_entangler_map()

In [6]:
nb_layer_application = 3

In [7]:
# Initializing observable
# diagonal = np.array(range(2**n_qubits_readout))
# avg_observable = np.zeros((2**n_qubits_readout, 2**n_qubits_readout))
# np.fill_diagonal(avg_observable, diagonal)
bitstring_values = list(range(2**n_qubits_readout))

In [8]:
@qml.qnode(dev)
def quantum_completer(weights, x=None):
    
    for i in range(nb_layer_application):
    
        encoding(x)
        variational_circuit(weights[2*i],weights[2*i+1])

    return prob(wires=list(range(n_qubits_readout))) 

In [9]:
# feature map 
# parameters should be of the form params = np.random.uniform(size=(num_layers, num_qubits, 3))

def encoding(x):
    for wire in range(n_wires // 3):
        qml.Rot(*x[3*wire: 3*(wire+1)], wires=[wire])     #Encode three features in each qubit

In [10]:
# QAOA ansatz definition
# unitary operator U_B with parameter beta
def U_B(beta):
    for wire in range(n_wires):
        qml.RX(2 * beta, wires=wire)

# unitary operator U_C with parameter gamma
def U_C(gamma):
    for wire_pair in graph:
        qml.CNOT(wires=[wire_pair[0], wire_pair[1]])
        qml.RZ(2*gamma, wires=wire_pair[1])
        qml.CNOT(wires=[wire_pair[0], wire_pair[1]])
        
def variational_circuit(gamma,beta):

    U_C(gamma)
    U_B(beta)

In [11]:
def loss(labels, predictions):
    return np.sum([(label - prediction)**2 for label, prediction in zip(labels, predictions)])

def cost(weights, X, Y):
    predictions = [np.mean(quantum_completer(weights, x=x) * bitstring_values) / 255. for x in X]
    return loss(Y, predictions)

In [12]:
%%time
# Define optimizer. 
opt = GradientDescentOptimizer(stepsize=0.4)

# Setup initial weights and number of optimization steps.
weights = np.random.random(2*nb_layer_application) 
steps = 30

# Optimizing the loss-function.
for it in range(3):
    # Update the weights by one optimizer step
    weights = opt.step(lambda v: cost(v, X, Y), weights)
    
    print("Iter: {:5d} | Cost: {:0.7f} ".format(it + 1, cost(weights, X, Y)))

Iter:     1 | Cost: 0.1682559 
Iter:     2 | Cost: 0.1681385 
Iter:     3 | Cost: 0.1680148 
CPU times: user 1min 49s, sys: 219 ms, total: 1min 49s
Wall time: 1min 41s
